# Orientation Analysis Plot (Polar coordinates)

## Imports

In [77]:
# Import required modules

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec

import seaborn as sns

import math

import re 

import pathlib
from pathlib import Path

import scipy
from scipy.signal import savgol_filter
from scipy.stats import circmean, circstd

import numpy as np

import pycircstat2
from pycircstat2.hypothesis import rao_spacing_test


plt.rcParams['svg.fonttype'] = 'none'

## Code

In [148]:
# Set matplotlib parameters for better visualization
mpl.rcParams['font.size'] = 25
mpl.rcParams['axes.labelsize'] = 25
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['legend.fontsize'] = 20

# Import data from filepath
folder_path = Path(r"C:\Users\mbgm4fs3\OneDrive - The University of Manchester\PhD\Experimental\Data\5. Mechanical Stimulation\Primary\Imaging\Pic Red\analysed")

# Create a DataFrame to store the Fourier analysis results
results_df = pd.DataFrame(columns=['Filename', 'NumPeaks', 'H2_Amplitude', 'H4_Amplitude', 
                                  'H4_to_H2_Ratio', 'DominantHarmonic', 
                                  'FirstHalfMeanAngle', 'FirstHalfNumPeaks'])

for csv_file in folder_path.glob("*.csv"):
    title = csv_file.stem
    print(f"Processing {title}...")

    # read in csv file
    df = pd.read_csv(csv_file)

    # Create a second copy shifted from [0, 180] → [180, 360]
    df_copy = df.assign(Orientation=df['Orientation'] + 180)     

    # append second half of circular data to first to get 360 degree data
    df_circular = pd.concat([df, df_copy], ignore_index=True)

    # now lets extend the data -180 and +180 to avoid artefacts when smoothing
    df_extended = pd.concat([df_circular.assign(Orientation=df_circular['Orientation'] - 360), 
                             df_circular, 
                             df_circular.assign(Orientation=df_circular['Orientation'] + 360)], 
                             ignore_index=True)

    # calculate orientation in radians
    df_extended['Orientation_rad'] = df_extended['Orientation'] * (math.pi / 180) 

    # normalise data 
    Slice1_min = df_extended['Slice1'].min()
    Slice1_max = df_extended['Slice1'].max()

    df_extended['Slice1_norm'] = (df_extended['Slice1'] - Slice1_min)/(Slice1_max - Slice1_min)

    # smooth signal
    df_extended['Slice1_smooth'] = savgol_filter(df_extended['Slice1_norm'], 50, 2)

    # Trim back to the original 0–360 range
    df_trimmed = df_extended[(df_extended['Orientation'] >= 0) & (df_extended['Orientation'] <= 360)].reset_index(drop=True)

    # -----------------------------------------
    # Fourier Analysis to assess the circular pattern
    # -----------------------------------------
    
    # Resample the signal to have uniform angular spacing for FFT
    angles = np.linspace(0, 2*np.pi, 360)
    # Interpolate the smoothed data to uniform angular spacing
    signal_interp = np.interp(angles, df_trimmed['Orientation_rad'], df_trimmed['Slice1_smooth'])
    
    # Apply Fourier Transform
    fft_result = np.fft.fft(signal_interp)
    fft_amplitude = np.abs(fft_result)
    
    # The 2nd harmonic corresponds to 2 cycles in 360 degrees (index 2 in FFT)
    # The 4th harmonic corresponds to 4 cycles in 360 degrees (index 4 in FFT)
    h2_amplitude = fft_amplitude[2] / len(angles)  # Normalize by signal length
    h4_amplitude = fft_amplitude[4] / len(angles)
    
    # Calculate ratio of 4th to 2nd harmonic
    h4_to_h2_ratio = h4_amplitude / h2_amplitude if h2_amplitude > 0 else float('inf')
    
    # Determine dominant harmonic
    dominant_harmonic = 4 if h4_amplitude > h2_amplitude else 2

    # -----------------------------------------
    # Store Results
    # -----------------------------------------
    
    # Store results
    results_df = pd.concat([results_df, pd.DataFrame({
        'Filename': [title],
        'NumPeaks': [num_peaks],
        'H2_Amplitude': [h2_amplitude],
        'H4_Amplitude': [h4_amplitude],
        'H4_to_H2_Ratio': [h4_to_h2_ratio],
        'DominantHarmonic': [dominant_harmonic],
        'OrientationAnglesMean': [first_half_mean_angle],
        'OrientaionPeaks': [first_half_num_peaks],
    })], ignore_index=True)

    # -----------------------------------------
    # Visualization
    # -----------------------------------------

    # re-bin data to plot rose plot 
    bin_size = 5

    # Assign each orientation to a bin
    df_trimmed['Angle_bin'] = (df_trimmed['Orientation'] // bin_size) * bin_size

    # Group by bin and aggregate (mean or sum, depending on what you want)
    rebinned = df_trimmed.groupby('Angle_bin')['Slice1_smooth'].mean().reset_index()

    # If you want it in radians for plotting
    rebinned['Angle_bin_rad'] = np.deg2rad(rebinned['Angle_bin'])

    # Min-max normalization
    min_val = rebinned['Slice1_smooth'].min()
    max_val = rebinned['Slice1_smooth'].max()

    rebinned['Slice1_smooth_norm'] = (rebinned['Slice1_smooth'] - min_val) / (max_val - min_val)

    # -----------------------------------------
    # Find Peaks in Binned Data and Calculate angles between peaks - ONLY FIRST HALF (0-180°)
    # -----------------------------------------
    

    # find peaks from data
    peaks, props = scipy.signal.find_peaks(rebinned['Slice1_smooth_norm'], distance=9, height=0.1)
    num_peaks = len(peaks)
    
    # -----------------------------------------
    # Calculate angles between peaks - ONLY FIRST HALF (0-180°)
    # -----------------------------------------
    peak_angles_deg = rebinned['Angle_bin'].iloc[peaks].values
    
    # Filter to only include peaks in the first half (0-180°)
    first_half_peaks = peak_angles_deg[(peak_angles_deg >= 0) & (peak_angles_deg <= 180)]
    first_half_peaks = np.sort(first_half_peaks)
    first_half_num_peaks = len(first_half_peaks)
    
    # Calculate angular differences between consecutive peaks in first half
    first_half_angle_diffs = []
    if first_half_num_peaks >= 2:
        for i in range(first_half_num_peaks-1):
            diff = first_half_peaks[i+1] - first_half_peaks[i]
            first_half_angle_diffs.append(diff)
        
        first_half_mean_angle = np.mean(first_half_angle_diffs)
    else:
        first_half_mean_angle = np.nan


    # Create a figure with 2 subplots: polar plot and FFT spectrum
    # fig = plt.figure(figsize=(21, 9))
    fig = plt.figure(figsize=(10, 10))

    # # adjust relative widths of each subplot
    # gs = gridspec.GridSpec(1, 2, width_ratios=[6, 1])  # wide : narrow

    
    # Polar plot for the circular data
    # need to use add_subplot as both plots require different axis (polar and cartisean)
    ax1 = fig.add_subplot(gs[0], projection='polar', facecolor='none')

    ax1.set_facecolor('none')

    ax1.bar(rebinned['Angle_bin_rad'],rebinned['Slice1_smooth_norm'], width=0.1, label='Normalised Data', color='teal', 
            edgecolor='black', linewidth=0.8, alpha=0.8)

    # Plot all peaks
    ax1.plot(rebinned['Angle_bin_rad'].iloc[peaks],
            rebinned['Slice1_smooth_norm'].iloc[peaks],
            'ro', label=f'Peaks')

    # Draw dashed line from peak to origin (0,0)
        # After identifying all your peaks
    for peak_idx in peaks:
        peak_theta = rebinned['Angle_bin_rad'].iloc[peak_idx]
        peak_r = rebinned['Slice1_smooth_norm'].iloc[peak_idx]
        ax1.plot([peak_theta, peak_theta], [0, peak_r], 'k--', linewidth=1.0) 


    # After plotting the dashed lines from peaks to origin
    # Filter peaks to only include those in the first half (0-180°)
    first_half_peaks = [idx for idx in peaks if 0 <= np.degrees(rebinned['Angle_bin_rad'].iloc[idx]) <= 180]
    first_half_peak_indices = sorted(first_half_peaks, key=lambda idx: rebinned['Angle_bin_rad'].iloc[idx])

    # For each pair of adjacent peaks in the first half
    for i in range(len(first_half_peak_indices) - 1):  # Note: -1 to avoid wrapping
        # Get current and next peak
        current_idx = first_half_peak_indices[i]
        next_idx = first_half_peak_indices[i + 1]
        
        # Get angles for current and next peak
        theta1 = rebinned['Angle_bin_rad'].iloc[current_idx]
        theta2 = rebinned['Angle_bin_rad'].iloc[next_idx]
        
        # Calculate radius for arc (use a fraction of the peak r value)
        arc_radius = 0.3 * min(rebinned['Slice1_smooth_norm'].iloc[current_idx], 
                            rebinned['Slice1_smooth_norm'].iloc[next_idx])
        
        # Generate points for the arc
        theta_arc = np.linspace(theta1, theta2, 50)
        r_arc = np.ones_like(theta_arc) * arc_radius
        
        # Plot the arc
        ax1.plot(theta_arc, r_arc, 'r-', alpha=0.7)

        # Calculate angle in degrees
        angle_deg = np.degrees(theta2 - theta1)
        
        # Add text label for the angle
        mid_theta = (theta1 + theta2) / 2
        text_radius = arc_radius * 1.7
        ax1.text(mid_theta, text_radius, f"{angle_deg:.1f}°", 
                ha='center', va='center', fontsize=10, 
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
        
    
    # ax1.set_title(f"{title}\nDominant Harmonic: {dominant_harmonic}")
    ax1.legend(loc='lower left', framealpha=1)
    # ax1.legend()
    ax1.grid(True)
    ax1.set_thetamin(0)
    ax1.set_thetamax(360)
    ax1.set_rmin(0)
    ax1.set_rmax(1.1)



    plt.tight_layout()
    plt.savefig(f'{title}.svg', format='svg', transparent=True, bbox_inches='tight')
    plt.close()

# Save the results to a CSV file
results_df.to_csv('harmonic_analysis_results.csv', index=False)

print("Analysis complete!")
print(f"Total samples analyzed: {len(results_df)}")

Processing w137 d14 400 con t6_OJ_Distribution...


C:\Users\mbgm4fs3\AppData\Local\Temp\ipykernel_19524\4029887444.py:79: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({


Processing w137 d14 800 con t6_OJ_Distribution...
Processing w137 d28 400 con tray 9_OJ_Distribution...
Processing w137 d28 400 exp tray 5_OJ_Distribution...
Processing w137 d28 800 con tray 5_OJ_Distribution...
Processing w137 d28 800 exp tray 4_OJ_Distribution...
Processing w164 d14 400 con t5_OJ_Distribution...
Processing w164 d14 800 con t6_OJ_Distribution...
Processing w164 d28 400 con tray 5_OJ_Distribution...
Processing w164 d28 400 exp tray 5_OJ_Distribution...
Processing w164 d28 800 con tray 6_OJ_Distribution...
Processing w164 d28 800 exp tray 5_OJ_Distribution...
Processing w184 d14 400 con t7_OJ_Distribution...
Processing w184 d14 800 con t7_OJ_Distribution...
Processing w184 d28 400 con tray 7_OJ_Distribution...
Processing w184 d28 400 exp tray 4_OJ_Distribution...
Processing w184 d28 800 con t8_OJ_Distribution...
Processing w184 d28 800 exp tray 5_OJ_Distribution...
Processing w184 d7 400 con t5_OJ_Distribution...
Processing w184 d7 800 con t5_OJ_Distribution...
Analysis

In [153]:
# Set matplotlib parameters for better visualization
mpl.rcParams['font.size'] = 25
mpl.rcParams['axes.labelsize'] = 25
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['legend.fontsize'] = 20

# Import data from filepath
folder_path = Path(r"C:\Users\mbgm4fs3\OneDrive - The University of Manchester\PhD\Experimental\Data\5. Mechanical Stimulation\Primary\Imaging\Pic Red\analysed")

# Function to find peaks with circular shifts to avoid starting point bias
def find_peaks_with_circular_shifts(signal, n_shifts=10, **peak_params):
    """Find peaks with multiple circular shifts to avoid starting point bias"""
    all_peaks = set()
    shift_size = len(signal) // n_shifts
    
    for i in range(n_shifts):
        # Shift the array circularly
        shifted_signal = np.roll(signal, i * shift_size)
        # Find peaks in shifted array
        shift_peaks, _ = scipy.signal.find_peaks(shifted_signal, **peak_params)
        # Convert peak indices back to original positions
        original_indices = (shift_peaks - i * shift_size) % len(signal)
        all_peaks.update(original_indices)
    
    return sorted(list(all_peaks))

# Create a DataFrame to store the Fourier analysis results
results_df = pd.DataFrame(columns=['Filename', 'NumPeaks', 'H2_Amplitude', 'H4_Amplitude', 
                                  'H4_to_H2_Ratio', 'DominantHarmonic', 
                                  'FirstHalfMeanAngle', 'FirstHalfNumPeaks'])

for csv_file in folder_path.glob("*.csv"):
    title = csv_file.stem
    print(f"Processing {title}...")

    # read in csv file
    df = pd.read_csv(csv_file)

    # Create a second copy shifted from [0, 180] → [180, 360]
    df_copy = df.assign(Orientation=df['Orientation'] + 180)     

    # append second half of circular data to first to get 360 degree data
    df_circular = pd.concat([df, df_copy], ignore_index=True)

    # now lets extend the data -180 and +180 to avoid artefacts when smoothing
    df_extended = pd.concat([df_circular.assign(Orientation=df_circular['Orientation'] - 360), 
                             df_circular, 
                             df_circular.assign(Orientation=df_circular['Orientation'] + 360)], 
                             ignore_index=True)

    # calculate orientation in radians
    df_extended['Orientation_rad'] = df_extended['Orientation'] * (math.pi / 180) 

    # normalise data 
    Slice1_min = df_extended['Slice1'].min()
    Slice1_max = df_extended['Slice1'].max()

    df_extended['Slice1_norm'] = (df_extended['Slice1'] - Slice1_min)/(Slice1_max - Slice1_min)

    # smooth signal
    df_extended['Slice1_smooth'] = savgol_filter(df_extended['Slice1_norm'], 50, 2)

    # Trim back to the original 0–360 range
    df_trimmed = df_extended[(df_extended['Orientation'] >= 0) & (df_extended['Orientation'] <= 360)].reset_index(drop=True)

    # -----------------------------------------
    # Fourier Analysis to assess the circular pattern
    # -----------------------------------------
    
    # Resample the signal to have uniform angular spacing for FFT
    angles = np.linspace(0, 2*np.pi, 360)
    # Interpolate the smoothed data to uniform angular spacing
    signal_interp = np.interp(angles, df_trimmed['Orientation_rad'], df_trimmed['Slice1_smooth'])
    
    # Apply Fourier Transform
    fft_result = np.fft.fft(signal_interp)
    fft_amplitude = np.abs(fft_result)
    
    # The 2nd harmonic corresponds to 2 cycles in 360 degrees (index 2 in FFT)
    # The 4th harmonic corresponds to 4 cycles in 360 degrees (index 4 in FFT)
    h2_amplitude = fft_amplitude[2] / len(angles)  # Normalize by signal length
    h4_amplitude = fft_amplitude[4] / len(angles)
    
    # Calculate ratio of 4th to 2nd harmonic
    h4_to_h2_ratio = h4_amplitude / h2_amplitude if h2_amplitude > 0 else float('inf')
    
    # Determine dominant harmonic
    dominant_harmonic = 4 if h4_amplitude > h2_amplitude else 2

    # -----------------------------------------
    # Visualization
    # -----------------------------------------

    # re-bin data to plot rose plot 
    bin_size = 5

    # Assign each orientation to a bin
    df_trimmed['Angle_bin'] = (df_trimmed['Orientation'] // bin_size) * bin_size

    # Group by bin and aggregate (mean or sum, depending on what you want)
    rebinned = df_trimmed.groupby('Angle_bin')['Slice1_smooth'].mean().reset_index()

    # If you want it in radians for plotting
    rebinned['Angle_bin_rad'] = np.deg2rad(rebinned['Angle_bin'])

    # Min-max normalization
    min_val = rebinned['Slice1_smooth'].min()
    max_val = rebinned['Slice1_smooth'].max()

    rebinned['Slice1_smooth_norm'] = (rebinned['Slice1_smooth'] - min_val) / (max_val - min_val)

    # -----------------------------------------
    # Find Peaks using the improved circular shifts method
    # -----------------------------------------
    
    # Use our improved peak finding function with circular shifts
    peaks = find_peaks_with_circular_shifts(rebinned['Slice1_smooth_norm'].values, 
                                           n_shifts=10, 
                                           distance=9, 
                                           height=0.1)
    
    num_peaks = len(peaks)
    
    # -----------------------------------------
    # Calculate angles between peaks - ONLY FIRST HALF (0-180°)
    # -----------------------------------------
    peak_angles_deg = rebinned['Angle_bin'].iloc[peaks].values
    
    # Filter to only include peaks in the first half (0-180°)
    first_half_peaks = peak_angles_deg[(peak_angles_deg >= 0) & (peak_angles_deg <= 180)]
    first_half_peaks = np.sort(first_half_peaks)
    first_half_num_peaks = len(first_half_peaks)
    
    # Calculate angular differences between consecutive peaks in first half
    first_half_angle_diffs = []
    if first_half_num_peaks >= 2:
        for i in range(first_half_num_peaks-1):
            diff = first_half_peaks[i+1] - first_half_peaks[i]
            first_half_angle_diffs.append(diff)
        
        first_half_mean_angle = np.mean(first_half_angle_diffs)
    else:
        first_half_mean_angle = np.nan

    # -----------------------------------------
    # Store Results
    # -----------------------------------------
    
    # Store results
    results_df = pd.concat([results_df, pd.DataFrame({
        'Filename': [title],
        'NumPeaks': [num_peaks],
        'H2_Amplitude': [h2_amplitude],
        'H4_Amplitude': [h4_amplitude],
        'H4_to_H2_Ratio': [h4_to_h2_ratio],
        'DominantHarmonic': [dominant_harmonic],
        'OrientationAnglesMean': [first_half_mean_angle],
        'OrientaionPeaks': [first_half_num_peaks],
    })], ignore_index=True)

    # Create a figure with transparent background
    fig = plt.figure(figsize=(10, 10), facecolor='none')
    
    # Use just one subplot that takes the entire figure
    ax1 = fig.add_subplot(111, projection='polar')
    ax1.set_facecolor('none')

    # Set width in radians for the bar chart
    width_in_radians = np.deg2rad(bin_size)

    # Plot the bars
    ax1.bar(rebinned['Angle_bin_rad'],
            rebinned['Slice1_smooth_norm'], 
            width=width_in_radians, 
            label='Normalised Data', 
            color='teal', 
            edgecolor='black', 
            linewidth=0.8, 
            alpha=0.8)

    # Plot all peaks
    ax1.plot(rebinned['Angle_bin_rad'].iloc[peaks],
            rebinned['Slice1_smooth_norm'].iloc[peaks],
            'ro', label=f'Peaks')

    # Draw dashed line from peak to origin (0,0)
    for peak_idx in peaks:
        peak_theta = rebinned['Angle_bin_rad'].iloc[peak_idx]
        peak_r = rebinned['Slice1_smooth_norm'].iloc[peak_idx]
        ax1.plot([peak_theta, peak_theta], [0, peak_r], 'k--', linewidth=1.0) 

    # After plotting the dashed lines from peaks to origin
    # Filter peaks to only include those in the first half (0-180°)
    first_half_peaks = [idx for idx in peaks if 0 <= np.degrees(rebinned['Angle_bin_rad'].iloc[idx]) <= 180]
    first_half_peak_indices = sorted(first_half_peaks, key=lambda idx: rebinned['Angle_bin_rad'].iloc[idx])

    # For each pair of adjacent peaks in the first half
    for i in range(len(first_half_peak_indices) - 1):  # Note: -1 to avoid wrapping
        # Get current and next peak
        current_idx = first_half_peak_indices[i]
        next_idx = first_half_peak_indices[i + 1]
        
        # Get angles for current and next peak
        theta1 = rebinned['Angle_bin_rad'].iloc[current_idx]
        theta2 = rebinned['Angle_bin_rad'].iloc[next_idx]
        
        # Calculate radius for arc (use a fraction of the peak r value)
        arc_radius = 0.3 * min(rebinned['Slice1_smooth_norm'].iloc[current_idx], 
                            rebinned['Slice1_smooth_norm'].iloc[next_idx])
        
        # Generate points for the arc
        theta_arc = np.linspace(theta1, theta2, 50)
        r_arc = np.ones_like(theta_arc) * arc_radius
        
        # Plot the arc
        ax1.plot(theta_arc, r_arc, 'r-', alpha=0.7)

        # Calculate angle in degrees
        angle_deg = np.degrees(theta2 - theta1)
        
        # Add text label for the angle
        mid_theta = (theta1 + theta2) / 2
        text_radius = arc_radius * 1.7
        ax1.text(mid_theta, text_radius, f"{angle_deg:.1f}°", 
                ha='center', va='center', fontsize=10, 
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
    
    # Format the plot
    ax1.legend(loc='lower left', framealpha=1)
    ax1.grid(True)
    ax1.set_thetamin(0)
    ax1.set_thetamax(360)
    ax1.set_rmin(0)
    ax1.set_rmax(1.1)

    plt.tight_layout()
    plt.savefig(f'{title}.svg', format='svg', transparent=True, bbox_inches='tight')
    plt.close()

# Save the results to a CSV file
results_df.to_csv('harmonic_analysis_results.csv', index=False)

print("Analysis complete!")
print(f"Total samples analyzed: {len(results_df)}")

Processing w137 d14 400 con t6_OJ_Distribution...


C:\Users\mbgm4fs3\AppData\Local\Temp\ipykernel_19524\446970735.py:151: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({


Processing w137 d14 800 con t6_OJ_Distribution...
Processing w137 d28 400 con tray 9_OJ_Distribution...
Processing w137 d28 400 exp tray 5_OJ_Distribution...
Processing w137 d28 800 con tray 5_OJ_Distribution...
Processing w137 d28 800 exp tray 4_OJ_Distribution...
Processing w164 d14 400 con t5_OJ_Distribution...
Processing w164 d14 800 con t6_OJ_Distribution...
Processing w164 d28 400 con tray 5_OJ_Distribution...
Processing w164 d28 400 exp tray 5_OJ_Distribution...
Processing w164 d28 800 con tray 6_OJ_Distribution...
Processing w164 d28 800 exp tray 5_OJ_Distribution...
Processing w184 d14 400 con t7_OJ_Distribution...
Processing w184 d14 800 con t7_OJ_Distribution...
Processing w184 d28 400 con tray 7_OJ_Distribution...
Processing w184 d28 400 exp tray 4_OJ_Distribution...
Processing w184 d28 800 con t8_OJ_Distribution...
Processing w184 d28 800 exp tray 5_OJ_Distribution...
Processing w184 d7 400 con t5_OJ_Distribution...
Processing w184 d7 800 con t5_OJ_Distribution...
Analysis